####This Notebook has UDF, generic framework - business functions & common functions

In [0]:
print("Running utility notebook to initialize all functions for further use")

##### UDF created to convert string to number word2number

In [0]:
%pip install word2number

In [0]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from word2number import w2n

def word_to_num(value):
    try:
        return(int(value))
    except:
        try:
            return w2n.word_to_num(value.lower())
        except:
            return None    
word_to_num_udf = udf(word_to_num, IntegerType())

###Business Logic

In [0]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window

def standardize_staff(df):
    return(df
           .withColumn("shipment_id",word_to_num_udf(F.col("shipment_id")).cast("long"))
           .withColumn("age",word_to_num_udf(F.col("age")).cast("int"))
           .withColumn("hub_location",F.initcap("hub_location"))
           .withColumn("role",F.lower("role"))
           .withColumn("loaddt",F.current_timestamp())
           .withColumn("origin_hub_city",F.initcap("hub_location"))
           .withColumn("fullname",F.concat_ws(" ","first_name","last_name"))
           .drop("first_name","last_name")
           .withColumnRenamed("fullname","staff_full_name")
    )

def scrub_geotag(df):
    return(df
        .withColumn("city_name",F.initcap("city_name"))
        .withColumn("masked_hub_location",F.initcap("country"))
    )
        
def standardize_shipments(df):
    return (df
        .withColumn("domain", F.lit("Logistics"))
        .withColumn("ingestion_timestamp", F.current_timestamp())
        .withColumn("is_expedited", F.lit(False).cast("boolean"))
        .withColumn("shipment_date", F.to_date("shipment_date", "yy-MM-dd"))
        .withColumn("shipment_cost", F.round("shipment_cost", 2))
        .withColumn("shipment_weight_kg", F.col("shipment_weight_kg").cast("double"))
    )

def enrich_shipments(df):
    return (df
        .withColumn("route_segment",
            F.concat_ws("-", "source_city", "destination_city"))
        .withColumn("vehicle_identifier",
            F.concat_ws("_", "vehicle_type", "shipment_id"))
        .withColumn("shipment_year", F.year("shipment_date"))
        .withColumn("shipment_month", F.month("shipment_date"))
        .withColumn("is_weekend",
            F.dayofweek("shipment_date").isin([1,7]))
        .withColumn("is_expedited",
            F.col("shipment_status").isin("IN_TRANSIT", "DELIVERED"))
        .withColumn("cost_per_kg",
            F.round(F.col("shipment_cost") / F.col("shipment_weight_kg"), 2))
        .withColumn("tax_amount",
            F.round(F.col("shipment_cost") * 0.18, 2))
        .withColumn("days_since_shipment",
            F.datediff(F.current_date(), "shipment_date"))
        .withColumn("is_high_value",
            F.col("shipment_cost") > 50000)
    )

def split_col(df):
    return(df
           .withColumn("order_prefix",F.substring("order_id",1,3))
           .withColumn("order_seq",F.substring("order_id",4,10))
           .withColumn("shipment_year",F.year("shipment_date"))
           .withColumn("shipment_month",F.month("shipment_date"))
           .withColumn("shipment_day",F.dayofmonth("shipment_date"))
           .withColumn("route_lane",F.concat_ws("->","source_city","destination_city"))
           )
    
def masked_name(col_name):
    return F.concat(
        F.substring(col_name,1,2),
        F.lit('*****'),
        F.substring(col_name,-1,1)
                    )



###Generic function

In [0]:
from pyspark.sql.session import SparkSession
def get_spark_session(appname="logistic usescases practice with FWs"):
    try:
        spark = SparkSession.getActiveSession()
        if spark:
            return spark
    except:
        pass    
    return (SparkSession.builder.config("spark.sql.shuffle.partitions","1").appName(appname).getOrCreate())


In [0]:
# read files functions

def read_csv_df(spark,path,header=True,infer_schema=True,sep=","):
    return_df =spark.read.option("header", header).\
                      option("infer_schema",infer_schema).\
                      option("sep",sep).\
                      csv(path)
    return return_df

def read_json_df(spark, path,mline=True):
    return spark.read.json(path,multiLine=mline,mode="PERMISSIVE")

def read_delta_df(spark,path):
    return spark.read.format("delta").load(path)

def read_file(spark,filetype,path,header=True,infer_schema=True,mline=True):
    if filetype=="csv":
        return spark.read.csv(path,header=header,inferSchema=infer_schema)#read_csv_df(spark,path)
    elif filetype=="json":
        return read_json_df(spark,path)
    elif filetype=="delta":
        return read_delta_df(spark,path)
    elif filetype=='orc':
        return spark.read.orc(path)
    else:
        raise Exception("File type not supported")

def read_table(spark,table_name):
    return spark.table(table_name)

In [0]:
#joindf
def join_df(df1,df2,how='inner', on='shipment_id'):
    return df1.join(df2, on=on, how=how)

def union_df(df1,df2):
    return df1.union(df2)

def unionDfSql(spark, view1, view2):
    return_df = spark.sql(f"select * from {view1} union select * from {view2}")
    return return_df

def mergeDf(df1,df2,allowmissingcol=True):
    return df1.unionByName(df2, allowMissingColumns=allowmissingcol)

In [0]:
from pyspark.sql.functions import lit
def add_literal_columns(df, columns, default_value=None):
    for col_name in columns:
        df = df.withColumn(col_name, lit(default_value))
        return df

In [0]:
#write operation
def write_file(df, path, mode="overwrite", format="delta"):
    return df.write.mode(mode).format(format).save(path)

def write_table(df, tablename, mode="overwrite"):
    df.write.mode(mode).format("delta").saveAsTable(tablename)   

In [0]:
def return_tempview_distinct_df(spark,tempview):
    return spark.sql(f"select distinct * from {tempview}")